In [1]:
! pip install git+https://github.com/xinyu1205/recognize-anything.git
from ram.models import ram
import numpy as np
import torch
from PIL import Image
from ram import inference_ram as inference
from ram import get_transform
from tqdm import tqdm
import pandas as pd
import os  # for checking file existence

  Cloning https://github.com/xinyu1205/recognize-anything.git to /tmp/pip-req-build-iuf4mkrw
  Running command git clone --filter=blob:none --quiet https://github.com/xinyu1205/recognize-anything.git /tmp/pip-req-build-iuf4mkrw
  Resolved https://github.com/xinyu1205/recognize-anything.git to commit 9a8f4b49075f6450dd44fbce0afb4c86d97867b6
  Preparing metadata (setup.py) ... - done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-h4yzpzzv/clip_142805bb79e140ad9984c7ede087829b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-h4yzpzzv/clip_142805bb79e140ad9984c7ede087829b
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... - \ done
  Using cached transformers-4.25.1-py3-none-any.whl.metadata (93 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transform

In [2]:
file_path = '../../datasets/CVQA/cvqa_RIC.csv'
# base_path = '/nfs/turbo/coe-mihalcea/shared_data/GD-VCR/X_VCR'
# base_path = '/nfs/turbo/coe-mihalcea/shared_data/GeoDE/images'
# base_path = '/nfs/turbo/coe-mihalcea/shared_data/dollarstreet'
base_path = '/nfs/turbo/coe-mihalcea/shared_data/cvqa/data/saved_images_RIC'
df = pd.read_csv(file_path)
display(df)

,ID,Subset,Question,Translated Question,Options,Translated Options,Label,Category,Image Type,Image Source,License,image_path
0,5920593854272599305_1,"('Tamil', 'India')",படத்தில் எத்தனை சுவைகள் காணப்படுகின்றன?,How many flavors of the dessert are seen in th...,['7' '3' '9' '5'],['7' '3' '9' '5'],-1,Cooking and food,External,https://commons.wikimedia.org/wiki/File:Faluda...,CC BY-SA 4.0,image_20.jpg
1,5865925794273878684_2,"('Romanian', 'Romania')",Din ce material sunt făcute bluzele din imagine?,What material are the blouses in the picture m...,['bumbac' 'mătase' 'lână' 'in'],['cotton' 'silk' 'wool' 'linen'],-1,Traditions / art / history,Self,Self-open,CC BY-SA,image_26.jpg
2,5865687724278338226_1,"('Bengali', 'India')",বাম পাশের লাল রঙের সব্জি টি কি?,What is the red coloured vegetable on the left...,['আলু' 'আদা' 'পেঁয়াজ' 'রসুন'],['Potato' 'Ginger' 'Onion' 'Garlic'],-1,Cooking and food,Self,Self-open,CC BY-SA,image_38.jpg
3,5865687724271272925_2,"('Bengali', 'India')",ডান পাশের সবুজ রঙের সব্জি টি কি?,What is the green coloured vegetable on the ri...,['পেঁপে' 'কুমড়া' 'সবুজ আপেল' 'পেয়ারা'],['Papaya' 'Pumpkin' 'Green Apple' 'Guava'],-1,Cooking and food,Self,Self-open,CC BY-SA,image_43.jpg
4,5865925804275165806_1,"('Romanian', 'Romania')",In ce sărbători se manâncă această mâncare?,In what holidays is this food eaten?,['Anul nou' 'Sfânta Maria' 'Crăciun' 'Paște'],['New Year' 'Saint Mary' 'Christmas' 'Easter'],-1,Cooking and food,Self,Self-open,CC BY-SA,image_44.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
122,5865687704279828011_0,"('Bengali', 'India')",ছবিতে দেখানো বস্তুটি কী?,What is the object shown in the image?,['কুলো' 'হাত পাখা' 'ছাঁকনি' 'খুন্তি'],[' winnowing fan' 'Handfan' 'Sieve' 'Spud'],-1,Objects / materials / clothing,Self,Self-research_only,CC BY-NC,image_961.jpg
123,5865925794275420071_1,"('Romanian', 'Romania')",În ce oraș este făcută poza?,In which city is the picture taken?,['Sibiu' 'Iași' 'București' 'Cluj-Napoca'],['Sibiu' 'Iasi' 'București' 'Cluj-Napoca'],-1,Geography / buildings / landmarks,Self,Self-open,CC BY-SA,image_978.jpg
124,5865925804271112509_0,"('Romanian', 'Romania')",Ce este in coșul din imagine?,What is in the basket in the image?,['roșii' 'conuri de brad vopsite' 'ouă vopsite...,['tomatoes' 'painted pine cones' 'painted eggs...,-1,Traditions / art / history,Self,Self-open,CC BY-SA,image_983.jpg
125,5865910684276112636_0,"('Chinese', 'China')",这是什么艺术？,Is this what you want?,['玩偶' '雕刻' '蜡像' '泥塑'],['Doll' 'Carving' 'Wax figure' 'Mud plastic'],-1,Traditions / art / history,External,https://www.flickr.com/photos/aaok/8443101631/...,Attribution-NonCommercial-NoDerivs License,image_994.jpg


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set image size and pretrained model path
image_size = 384
pretrained_model_path = 'recognize-anything/pretrained/ram_swin_large_14m.pth'  # Update this path to your model

# Get the transform function
transform = get_transform(image_size=image_size)

# Load the model
model = ram(pretrained=pretrained_model_path, image_size=image_size, vit='swin_l')
model.eval()
model = model.to(device)

def get_formatted_tags(image_path):
    """
    Process an image and return formatted tags.

    Args:
    image_path (str): Path to the image file.

    Returns:
    str: Formatted tags separated by commas.
    """

    if not os.path.exists(image_path):
        print(f"File not found: {image_path}")
        return None


    # Load and transform the image
    image = transform(Image.open(image_path)).unsqueeze(0).to(device)

    # Perform inference
    tags = inference(image, model)

    # Format the English tags by replacing ' | ' with ', '
    english_tags = tags[0].replace(' | ', ', ')

    return english_tags

/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
recognize-anything/pretrained/ram_swin_large_14m.pth
--------------
load checkpoint from recognize-anything/pretrained/ram_swin_large_14m.pth
vit: swin_l


In [8]:
tqdm.pandas()
df['tag'] = df['image_path'].progress_apply(lambda x: get_formatted_tags(base_path + '/' + x))

df = df.dropna(subset=['tag'])

# Display the updated DataFrame and save it if needed
display(df)

# Optionally save the DataFrame back to CSV
df.to_csv('dollarstreet_RIC_tagged.csv', index=False)

100%|██████████| 127/127 [00:28<00:00,  4.49it/s]


,ID,Subset,Question,Translated Question,Options,Translated Options,Label,Category,Image Type,Image Source,License,image_path,tag
0,5920593854272599305_1,"('Tamil', 'India')",படத்தில் எத்தனை சுவைகள் காணப்படுகின்றன?,How many flavors of the dessert are seen in th...,['7' '3' '9' '5'],['7' '3' '9' '5'],-1,Cooking and food,External,https://commons.wikimedia.org/wiki/File:Faluda...,CC BY-SA 4.0,image_20.jpg,"beverage, bottle, bubble tea, case, color, con..."
1,5865925794273878684_2,"('Romanian', 'Romania')",Din ce material sunt făcute bluzele din imagine?,What material are the blouses in the picture m...,['bumbac' 'mătase' 'lână' 'in'],['cotton' 'silk' 'wool' 'linen'],-1,Traditions / art / history,Self,Self-open,CC BY-SA,image_26.jpg,"case, clothing, costume, display, dress, garme..."
2,5865687724278338226_1,"('Bengali', 'India')",বাম পাশের লাল রঙের সব্জি টি কি?,What is the red coloured vegetable on the left...,['আলু' 'আদা' 'পেঁয়াজ' 'রসুন'],['Potato' 'Ginger' 'Onion' 'Garlic'],-1,Cooking and food,Self,Self-open,CC BY-SA,image_38.jpg,"basket, lamp, floor, garlic, man, market, onio..."
3,5865687724271272925_2,"('Bengali', 'India')",ডান পাশের সবুজ রঙের সব্জি টি কি?,What is the green coloured vegetable on the ri...,['পেঁপে' 'কুমড়া' 'সবুজ আপেল' 'পেয়ারা'],['Papaya' 'Pumpkin' 'Green Apple' 'Guava'],-1,Cooking and food,Self,Self-open,CC BY-SA,image_43.jpg,"array, eggplant, avocado, bell pepper, carrot,..."
4,5865925804275165806_1,"('Romanian', 'Romania')",In ce sărbători se manâncă această mâncare?,In what holidays is this food eaten?,['Anul nou' 'Sfânta Maria' 'Crăciun' 'Paște'],['New Year' 'Saint Mary' 'Christmas' 'Easter'],-1,Cooking and food,Self,Self-open,CC BY-SA,image_44.jpg,"bread, cheese, counter top, couple, table, foo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,5865687704279828011_0,"('Bengali', 'India')",ছবিতে দেখানো বস্তুটি কী?,What is the object shown in the image?,['কুলো' 'হাত পাখা' 'ছাঁকনি' 'খুন্তি'],[' winnowing fan' 'Handfan' 'Sieve' 'Spud'],-1,Objects / materials / clothing,Self,Self-research_only,CC BY-NC,image_961.jpg,"cloth, fan, pink, sunshade, umbrella"
123,5865925794275420071_1,"('Romanian', 'Romania')",În ce oraș este făcută poza?,In which city is the picture taken?,['Sibiu' 'Iași' 'București' 'Cluj-Napoca'],['Sibiu' 'Iasi' 'București' 'Cluj-Napoca'],-1,Geography / buildings / landmarks,Self,Self-open,CC BY-SA,image_978.jpg,"building, car, church, city, city street, clou..."
124,5865925804271112509_0,"('Romanian', 'Romania')",Ce este in coșul din imagine?,What is in the basket in the image?,['roșii' 'conuri de brad vopsite' 'ouă vopsite...,['tomatoes' 'painted pine cones' 'painted eggs...,-1,Traditions / art / history,Self,Self-open,CC BY-SA,image_983.jpg,"basket, draw, dye, easter, easter egg, egg, fi..."
125,5865910684276112636_0,"('Chinese', 'China')",这是什么艺术？,Is this what you want?,['玩偶' '雕刻' '蜡像' '泥塑'],['Doll' 'Carving' 'Wax figure' 'Mud plastic'],-1,Traditions / art / history,External,https://www.flickr.com/photos/aaok/8443101631/...,Attribution-NonCommercial-NoDerivs License,image_994.jpg,"bust, clay, face, head, person, man, mud, scul..."
